In [1]:
import cv2
import numpy as np
import math
import random

In [2]:
img = cv2.imread('/home/ayush/My Projects/Number Plate/1.png')

In [3]:
class PossibleChar:
    def __init__(self, contour):
        self.contour = contour
        self.rect = cv2.boundingRect(self.contour)
        [X, Y, width, height] = self.rect
        self.X = X
        self.Y = Y
        self.width = width
        self.height = height
        
        self.area = self.width * self.height
        
        self.centerX = self.X + (float)(self.width)/2
        self.centerY = self.Y + (float)(self.height)/2
        
        self.diagDist = math.sqrt(self.height**2 + self.width**2)
        
        self.aspect_ratio = float(self.width)/float(self.height)
        

In [4]:
def display(img, name):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [5]:
def preprocess(img):
    ### HSV SHOWS BRIHGTENED REGIONS BETTER THAN GRAY
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    imgHue, imgSat, imgValue = cv2.split(hsv)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    tophat = cv2.morphologyEx(imgValue, cv2.MORPH_TOPHAT, kernel)
    blackhat = cv2.morphologyEx(imgValue, cv2.MORPH_BLACKHAT, kernel)
    top_plus_black = cv2.add(imgValue, tophat)
    top_plus_black_minus_black = cv2.subtract(top_plus_black, blackhat)

    blur = cv2.GaussianBlur(top_plus_black_minus_black, (5, 5), 0)
    thresh = cv2.adaptiveThreshold(blur, 255.0, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 19, 9)
#     display(thresh, "Thresholded Image")
    return imgValue, thresh

In [6]:
imgGray, imgThresh = preprocess(img)

In [7]:
imgThreshCopy = imgThresh.copy()
img2, contours, hierarchy = cv2.findContours(imgThreshCopy, cv2.RETR_LIST , cv2.CHAIN_APPROX_SIMPLE)

In [8]:
MIN_AREA = 100
MIN_WIDTH = 4
MIN_HEIGHT = 8
MIN_ASPECT_RATIO = 0.25
MAX_ASPECT_RATIO = 1.0
def detect_chars(imgThresh):
    possiblechars = []
    imgContour = np.zeros((imgThresh.shape[0], imgThresh.shape[1], 3), np.uint8)
    for i in range(len(contours)):
    #     cv2.drawContours(imgContour, contours, i, (255.0, 255.0, 255.0))
        possibleChar = PossibleChar(contours[i])
        if (possibleChar.area>MIN_AREA and possibleChar.width>MIN_WIDTH and possibleChar.height>MIN_HEIGHT and 
            possibleChar.aspect_ratio>MIN_ASPECT_RATIO and possibleChar.aspect_ratio<MAX_ASPECT_RATIO):
            
            cv2.drawContours(imgContour, contours, i, (255.0, 255.0, 255.0))
            possiblechars.append(possibleChar)
            
    display(imgContour, "{}".format(i))
    return possiblechars

In [9]:
possiblechars = detect_chars(imgThresh)

In [12]:
def findListOfListsOfMatchingChars(listOfPossibleChars):

    listOfListsOfMatchingChars = []

    for possibleChar in listOfPossibleChars:
        listOfMatchingChars = []

        for possibleMatchingChar in listOfPossibleChars:
            if possibleMatchingChar == possibleChar:
                continue
            fltDistanceBetweenChars = distanceBetweenChars(possibleChar, possibleMatchingChar)

            fltAngleBetweenChars = angleBetweenChars(possibleChar, possibleMatchingChar)

            fltChangeInArea = float(abs(possibleMatchingChar.area - possibleChar.area)) / float(possibleChar.area)

            fltChangeInWidth = float(abs(possibleMatchingChar.width - possibleChar.width)) / float(possibleChar.width)
            fltChangeInHeight = float(abs(possibleMatchingChar.height - possibleChar.height)) / float(possibleChar.height)

            if (fltDistanceBetweenChars < (possibleChar.diagDist * 5.0) and
                fltAngleBetweenChars < 12.0 and
                fltChangeInArea < 0.5 and
                fltChangeInWidth < 0.8 and
                fltChangeInHeight < 0.2):

                listOfMatchingChars.append(possibleMatchingChar)

        listOfMatchingChars.append(possibleChar)

        if len(listOfMatchingChars) < 3:
            continue
        
        listOfListsOfMatchingChars.append(listOfMatchingChars)

        listOfPossibleChars = list(set(listOfPossibleChars) - set(listOfMatchingChars))

    return listOfListsOfMatchingChars


def distanceBetweenChars(firstChar, secondChar):
    intX = abs(firstChar.centerX - secondChar.centerX)
    intY = abs(firstChar.centerY - secondChar.centerY)
    return math.sqrt((intX ** 2) + (intY ** 2))


def angleBetweenChars(firstChar, secondChar):
    fltAdj = float(abs(firstChar.centerX - secondChar.centerX))
    fltOpp = float(abs(firstChar.centerY - secondChar.centerY))
    if fltAdj != 0.0:
        fltAngleInRad = math.atan(fltOpp / fltAdj)
    else:
        fltAngleInRad = 1.5708
    fltAngleInDeg = fltAngleInRad * (180.0 / math.pi)
    return fltAngleInDeg

In [13]:
listlistMatchingChars = findListOfListsOfMatchingChars(possiblechars)
print len(listlistMatchingChars)

15


In [17]:
imgContours = np.zeros((imgThresh.shape[0], imgThresh.shape[1], 3), np.uint8)

for listOfMatchingChars in listlistMatchingChars:
    contours = []
    for matchingChar in listOfMatchingChars:
        contours.append(matchingChar.contour)
        cv2.drawContours(imgContours, contours, -1, (255.0, 255.0, 255.0))

display(imgContours, "Reduced Image")